# Brochure Builder – Full Business Solution

## Project Overview
This project extends the Day 1 prototype into a complete business-ready solution.  
Given a company name and its primary website, the system automatically generates a polished brochure suitable for prospective clients, investors, and potential recruits.

## Business Challenge
Design and implement a product that:
- Ingests a company’s main website.
- Extracts and organizes key information about the business.
- Produces a clear, compelling brochure that non-technical stakeholders can use.

## Real-World Use
This solution can be adapted for:
- Sales and marketing collateral.
- Investor pitch materials.
- Employer branding and recruiting assets.


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper_new import fetch_website_links_dynamic, fetch_website_contents_dynamic
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'  # Fixed: was 'gpt-5-nano'
openai = OpenAI()


API key looks good so far


In [3]:
links = fetch_website_links_dynamic("https://spearmint.online")
content = fetch_website_contents_dynamic("https://spearmint.online")


In [4]:
links


['tel:+9187560 80645',
 'mailto:info@spearmint.online',
 'tel:+9187560 80645',
 'mailto:info@spearmint.online',
 'https://spearmint.online/',
 'https://spearmint.online/',
 'https://spearmint.online',
 'https://spearmint.online/web-design/',
 'https://spearmint.online/reactjs-development/',
 'https://spearmint.online/next-js-development/',
 'https://spearmint.online/node-js-development/',
 'https://spearmint.online/angular-js-development/',
 'https://spearmint.online/python-development/',
 'https://spearmint.online',
 'https://spearmint.online/cross-platform/',
 'https://spearmint.online/app-designing/',
 'https://spearmint.online/native-apps/',
 'https://spearmint.online/wearables-app/',
 'https://spearmint.online/progressive-web-app/',
 'https://spearmint.online/ott-apps/',
 'https://spearmint.online',
 'https://spearmint.online/smart-contracts-development/',
 'https://spearmint.online/smart-contracts-audit/',
 'https://spearmint.online/web3-development-services/',
 'https://spearmin

In [5]:
content

'Blockchain Development Services | Spearmint\n\n+9187560 80645\ninfo@spearmint.online\n+9187560 80645\ninfo@spearmint.online\nHOME\nWeb\nWeb Designing\nReact JS Development\nNext JS Development\nNode JS Development\nAngular JS Development\nPython Development\nMobile\nCross Platform\nApp Designing\nNative Apps\nWearable Apps\nProgressive Web App\nOTT Apps\nBlockchain\nSmart Contract Development\nSmart Contract Audit\nWeb3 Development\nToken Development\nToken Development Company\nBlockchain Social Media Development\nServices\nWeb Application\nData Science\nMachine Learning Development\nIoT Development\nDevOps Services\nSaaS Development Maintenance\nDigital Marketing\nWhitepaper Writing\nSOCIAL MEDIA MARKETING (SMM)\nSOCIAL MEDIA OPTIMIZATION (SMO)\nPAY PER CLICK (PPC)\nSearch Engine Optimization (SEO)\nInfluencer Marketing\nGrowth & Performance Marketing\nVoice & AI Marketing\nVideo & Youtube Marketing\nInfluencer & Affiliate Marketing\nLead Generation\nConversion Rate Optimization\nMar

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    """Generate prompt with links from spearmint.online"""
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, tel: or mailto: links.

Links (some might be relative links):

"""
    # Use your dynamic scraper instead of fetch_website_links
    links = fetch_website_links_dynamic(url)
    user_prompt += "\n".join(links)
    return user_prompt


In [8]:
print(get_links_user_prompt("https://spearmint.online"))



Here is the list of links on the website https://spearmint.online -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links, tel: or mailto: links.

Links (some might be relative links):

tel:+9187560 80645
mailto:info@spearmint.online
tel:+9187560 80645
mailto:info@spearmint.online
https://spearmint.online/
https://spearmint.online/
https://spearmint.online
https://spearmint.online/web-design/
https://spearmint.online/reactjs-development/
https://spearmint.online/next-js-development/
https://spearmint.online/node-js-development/
https://spearmint.online/angular-js-development/
https://spearmint.online/python-development/
https://spearmint.online
https://spearmint.online/cross-platform/
https://spearmint.online/app-designing/
https://spearmint.online/native-apps/
https://spearmint.online/wearables-app/
https://spearmint.online/progressive-web-app/
https:

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links


In [10]:
# Get filtered, relevant links for brochure
relevant_links = select_relevant_links("https://spearmint.online")
print(json.dumps(relevant_links, indent=2))


Selecting relevant links for https://spearmint.online by calling gpt-4o-mini
Found 5 relevant links
{
  "links": [
    {
      "type": "homepage",
      "url": "https://spearmint.online/"
    },
    {
      "type": "about page",
      "url": "https://spearmint.online/about-us/"
    },
    {
      "type": "careers page",
      "url": "https://spearmint.online/careers/"
    },
    {
      "type": "contact page",
      "url": "https://spearmint.online/contact-us/"
    },
    {
      "type": "services page",
      "url": "https://spearmint.online/services/"
    }
  ]
}


In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4o-mini
Found 6 relevant links


{'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs'}]}

In [12]:
# Step 1: Fetch Page and Relevant Links (with rate limiting)

import time

def fetch_page_and_all_relevant_links(url, max_links=3):
    """Fetch main page and limited relevant links with rate limiting"""
    print(f"Fetching main page: {url}")
    contents = fetch_website_contents_dynamic(url)
    
    print(f"Selecting relevant links...")
    relevant_links = select_relevant_links(url)
    
    # Limit to first N links to avoid browser crashes
    links_to_fetch = relevant_links['links'][:max_links]
    print(f"Fetching {len(links_to_fetch)} most relevant pages...")
    
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    
    for i, link in enumerate(links_to_fetch):
        print(f"Fetching {i+1}/{len(links_to_fetch)}: {link['type']}")
        try:
            result += f"\n\n### Link: {link['type']}\n"
            result += fetch_website_contents_dynamic(link["url"])
            time.sleep(2)  # 2 second delay between requests
        except Exception as e:
            print(f"Error fetching {link['url']}: {e}")
            result += f"[Could not fetch this page]\n"
    
    return result


In [13]:
# Step 2: Test the Fetcher
print(fetch_page_and_all_relevant_links("https://spearmint.online"))


Fetching main page: https://spearmint.online
Selecting relevant links...
Selecting relevant links for https://spearmint.online by calling gpt-4o-mini
Found 5 relevant links
Fetching 3 most relevant pages...
Fetching 1/3: home page
Error fetching https://spearmint.online/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=143.0.7499.40)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7617a8255
	0x7ff7617a82b0
	0x7ff76158165d
	0x7ff76157e1dd
	0x7ff76156eb69
	0x7ff761570974
	0x7ff76156f10f
	0x7ff76156e871
	0x7ff76156e5bd
	0x7ff76156c20a
	0x7ff76156c9f2
	0x7ff761585950
	0x7ff76162bad4
	0x7ff761601fda
	0x7ff76162ac97
	0x7ff7615cac29
	0x7ff7615cba93
	0x7ff761abfff0
	0x7ff761aba930
	0x7ff761ad9096
	0x7ff7617c5754
	0x7ff7617ce6fc
	0x7ff7617b1974
	0x7ff7617b1b25
	0x7ff761797852
	0x7fff185b7374
	0x7fff19ebcc91

Fetching 2/3: about page
Fetching 3/3: services page
## Landing Page:

Blockchain Development Services | Spearmint

+9187560 80645
info@

In [14]:
# Step 3: Define Brochure Prompts
# Professional version
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Optional: Humorous version
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
# Step 4: Generate User Prompt
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt


In [16]:
# Step 5: Create Brochure (Non-Streaming)
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,  # Uses your gpt-4o-mini
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Generate brochure
create_brochure("Spearmint", "https://spearmint.online")

Fetching main page: https://spearmint.online
Selecting relevant links...
Selecting relevant links for https://spearmint.online by calling gpt-4o-mini
Found 4 relevant links
Fetching 3 most relevant pages...
Fetching 1/3: about page
Error fetching https://spearmint.online/about-us/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=143.0.7499.40)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7617a8255
	0x7ff7617a82b0
	0x7ff76158165d
	0x7ff76157e1dd
	0x7ff76156eb69
	0x7ff761570974
	0x7ff76156f10f
	0x7ff76156e871
	0x7ff76156e5bd
	0x7ff76156c20a
	0x7ff76156c9f2
	0x7ff761585950
	0x7ff76162bad4
	0x7ff761601fda
	0x7ff76162ac97
	0x7ff7615cac29
	0x7ff7615cba93
	0x7ff761abfff0
	0x7ff761aba930
	0x7ff761ad9096
	0x7ff7617c5754
	0x7ff7617ce6fc
	0x7ff7617b1974
	0x7ff7617b1b25
	0x7ff761797852
	0x7fff185b7374
	0x7fff19ebcc91

Fetching 2/3: careers page
Fetching 3/3: contact page


# Spearmint Company Brochure

## Overview

**Spearmint** is a cutting-edge technology company specializing in blockchain development, mobile applications, and comprehensive digital marketing solutions. With a focus on innovation and quality, Spearmint caters to diverse industries, delivering tailored solutions that drive growth and efficiency for businesses.

## Services Offered

### Blockchain Development Services
- **Smart Contract Development**: Creating decentralized contracts to automate and secure transactions.
- **Web3 Development**: Building next-generation web applications utilizing blockchain technology.
- **Token Development**: Crafting unique tokens for various applications, including Initial Coin Offerings (ICOs).
- **Blockchain Social Media Development**: Developing decentralized social media platforms.

### Mobile App Development
- **Cross-Platform Apps**: Design and development across multiple platforms to reach a broader audience.
- **Native Apps**: High-performance mobile apps tailored for specific platforms.
- **Progressive Web Apps (PWAs)**: Web applications that offer a native app-like experience.
- **Wearable Apps & OTT Apps**: Specialized apps for wearable technology and streaming services.

### Web Development
- **Web Designing**: Creating user-friendly websites that reflect the brand’s identity.
- **Framework Development**: Expert development using React JS, Next JS, Node JS, and Angular JS.

### Digital Marketing Solutions
- **Search Engine Optimization (SEO)**: Enhancing online visibility for better organic search results.
- **Social Media Marketing (SMM)**: Engaging audiences on various social platforms to boost brand awareness.
- **Influencer Marketing**: Collaborating with key influencers to reach targeted demographics.
- **Lead Generation & Conversion Rate Optimization**: Implementing strategies to increase customer acquisition and sales efficiency.

## Company Culture

At Spearmint, we believe in fostering a collaborative and innovative environment. Our team is composed of forward-thinkers who are passionate about technology and dedicated to helping clients succeed. We value creativity, transparency, and continuous learning, making us a great place to grow your career.

## Careers at Spearmint

Spearmint offers exciting career opportunities in software development, digital marketing, and project management, particularly in the fields of blockchain and artificial intelligence. We are always looking for talented individuals who are eager to make an impact and innovate in a fast-paced environment. Our remote work culture provides flexibility while promoting a healthy work-life balance.

## Our Customers

We serve a diverse clientele, including startups, established enterprises, and organizations exploring blockchain technology and digital transformation. Our customer-centric approach ensures that we deliver solutions tailored to unique business needs, helping our clients achieve their objectives efficiently.

## Contact Us

If you are interested in our services or would like to explore career opportunities, feel free to reach out to us:

- **Phone**: +9187560 80645
- **Email**: info@spearmint.online

Join us at Spearmint as we define the future of technology and innovation!

In [17]:
# Step 6: Streaming Version (Typewriter Effect)
def stream_brochure(company_name, url):
    """Stream brochure with typewriter animation"""
    stream = openai.chat.completions.create(
        model=MODEL,  # Uses your gpt-4o-mini
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

# Generate streaming brochure
stream_brochure("Spearmint", "https://spearmint.online")


Fetching main page: https://spearmint.online
Selecting relevant links...
Selecting relevant links for https://spearmint.online by calling gpt-4o-mini
Found 5 relevant links
Fetching 3 most relevant pages...
Fetching 1/3: home page
Error fetching https://spearmint.online/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=143.0.7499.40)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7617a8255
	0x7ff7617a82b0
	0x7ff76158165d
	0x7ff76157e1dd
	0x7ff76156eb69
	0x7ff761570974
	0x7ff76156f10f
	0x7ff76156e871
	0x7ff76156e5bd
	0x7ff76156c20a
	0x7ff76156c9f2
	0x7ff761585950
	0x7ff76162bad4
	0x7ff761601fda
	0x7ff76162ac97
	0x7ff7615cac29
	0x7ff7615cba93
	0x7ff761abfff0
	0x7ff761aba930
	0x7ff761ad9096
	0x7ff7617c5754
	0x7ff7617ce6fc
	0x7ff7617b1974
	0x7ff7617b1b25
	0x7ff761797852
	0x7fff185b7374
	0x7fff19ebcc91

Fetching 2/3: about page
Fetching 3/3: careers page


# Welcome to Spearmint Technologies

## Transforming Ideas with Cutting-Edge Solutions

Spearmint is a prominent player in the technology landscape, specializing in **Blockchain Development Services**, **Web Development**, **Mobile Applications**, and **Digital Marketing**. With a commitment to innovation and quality, we help businesses leverage modern technologies to achieve their goals.

### Our Expertise

- **Web Development**:
  - Web Designing
  - React, Next, Node, Angular, and Python Development

- **Mobile Applications**:
  - Cross Platform Apps 
  - Native and Wearable Apps 
  - Progressive Web Apps 
  - OTT Apps 

- **Blockchain Solutions**:
  - Smart Contract Development & Audits
  - Web3 Development
  - Token Development 
  - Blockchain Social Media Development 

- **Digital Marketing**:
  - Search Engine Optimization (SEO)
  - Social Media Marketing (SMM)
  - Pay Per Click (PPC)
  - Influencer & Affiliate Marketing
  - Lead Generation and more!

## Our Culture

At Spearmint, we believe that our success stems from our outstanding team and vibrant company culture. We foster an environment of creativity, collaboration, and continuous learning, where each team member is encouraged to innovate and take ownership of their projects. Our diverse workforce thrives on shared knowledge, pushing the boundaries of what is possible in the tech industry.

### Why Join Us?

We're not just a workplace; we are a community of passionate individuals driving technological advancements. By joining Spearmint, you will have access to:
- **Remote Work Opportunities**: Work from anywhere while maintaining a healthy work-life balance.
- **Career Growth**: Extensive development programs to enhance your skills and career trajectory.
- **Innovative Projects**: Opportunity to work on groundbreaking projects in fields like AI, Crypto, and IoT.

## Our Clients

We are proud to partner with businesses of all sizes, from startups looking to establish their digital presence to established enterprises aiming to innovate through technology. Our strategic approach and tailored solutions make us an ideal partner to navigate the digital landscape successfully.

### Get in Touch

Ready to transform your ideas into reality? Contact us today!
- **Phone**: +9187560 80645
- **Email**: info@spearmint.online

Explore the possibilities with Spearmint Technologies—where your vision meets our expertise!

Save the Brochure

In [18]:
def save_brochure(company_name, url, filename="brochure.md"):
    """Create and save brochure to file"""
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(result)
    
    print(f"✅ Brochure saved to {filename}")
    display(Markdown(result))

save_brochure("Spearmint", "https://spearmint.online", "spearmint_brochure.md")


Fetching main page: https://spearmint.online
Selecting relevant links...
Selecting relevant links for https://spearmint.online by calling gpt-4o-mini
Found 5 relevant links
Fetching 3 most relevant pages...
Fetching 1/3: home page
Fetching 2/3: about page
Fetching 3/3: careers page
✅ Brochure saved to spearmint_brochure.md


# Spearmint Technologies Brochure

## Who We Are  
Spearmint is a leading provider of blockchain development services and digital solutions. Our expertise spans across web and mobile app development, data science, machine learning, and digital marketing. We leverage cutting-edge technologies to help businesses transform and thrive in an ever-evolving digital landscape.

## Our Services  
At Spearmint, we offer a comprehensive suite of services, including:

- **Web Development**  
  - **Web Design**  
  - **React JS, Next JS, Angular JS, Node JS & Python Development**  
- **Mobile App Development**  
  - **Cross-Platform, Native, Wearable, and Progressive Web Apps**  
- **Blockchain Solutions**  
  - **Smart Contract Development & Audits**  
  - **Web3 Development & Token Development**  
  - **Blockchain Social Media Development**  
- **Data Science & AI**  
  - **Machine Learning Development & IoT Solutions**  
- **Digital Marketing**  
  - **SEO, PPC, SMM & Social Media Optimization**  
  - **Content Marketing, Influencer Marketing & Lead Generation**  

## Our Customers  
We cater to a diverse clientele, from startups looking to establish a digital presence to established enterprises seeking to innovate and enhance their operations through technology. Our custom solutions are tailored to meet the unique needs of each client, ensuring they achieve their business objectives seamlessly.

## Company Culture  
At Spearmint, we pride ourselves on fostering a collaborative and innovative work environment. Our team is made up of passionate professionals who thrive on creativity, teamwork, and continuous learning. We encourage an open dialogue where everyone’s ideas are valued and new solutions are born from collective thinking. We believe in balancing hard work with a supportive atmosphere.

## Careers at Spearmint  
Join us at Spearmint! We are always on the lookout for talented individuals who are eager to make an impact in the tech industry. If you are passionate about technology and innovation, and want to work in a dynamic team, reach out to us. Explore career opportunities that resonate with your skills and aspirations!

## Get in Touch  
For inquiries or to learn more about how we can help you, please contact us:
- **Phone:** +9187560 80645
- **Email:** info@spearmint.online

Discover the future of technology with Spearmint. Unlock your potential today!